In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision import models
from torchsummary import summary
import wandb
import os

In [ ]:
# =======================
# STEP 0: Initialize wandb
# =======================
wandb.init(project="ResNet-flowers", config={
    "epochs": 50,
    "batch_size": 16,
    "learning_rate": 0.001,
    "architecture": "ResNet",
    "pretrained": True,
    "input_size": 224
})

# Shortcut to config values
config = wandb.config

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vizuara-info (pritkudale-vizuara) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# =======================
# STEP 1: Data Preparation
# =======================

# Transforms for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

train_dir = "/content/drive/MyDrive/5flowersdata/flowers/train"
val_dir = "/content/drive/MyDrive/5flowersdata/flowers/val"

train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=val_dir, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size)

In [ ]:
# ===========================
# STEP 2: Load Pretrained Model
# ===========================
from torchvision.models import ResNet50_Weights

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 5)  # for 5-class classification

# Freeze all but final layer
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Watch the model's weights and gradients
wandb.watch(model, log="all", log_freq=10)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 162MB/s]


In [ ]:
# ===================
# STEP 3: Loss & Optimizer
# ===================

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_correct = 0
        train_total = 0
        running_loss = 0.0

        print(f"\nEpoch {epoch + 1}/{epochs}")
        print("-" * 30)

        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            batch_correct = (preds == labels).sum().item()
            train_correct += batch_correct
            train_total += labels.size(0)

            # Print every 10 batches
            if (i + 1) % 10 == 0:
                batch_acc = batch_correct / labels.size(0)
                print(f"[Batch {i+1}/{len(train_loader)}] Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")

        train_acc = train_correct / train_total
        wandb.log({"epoch": epoch + 1, "train_loss": running_loss, "train_accuracy": train_acc})
        print(f"Epoch {epoch+1} Summary - Loss: {running_loss:.4f}, Train Accuracy: {train_acc:.4f}")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total
        wandb.log({"epoch": epoch + 1, "val_accuracy": val_acc})
        print(f"Validation Accuracy: {val_acc:.4f}")


In [ ]:
# ===================
# Train the model
# ===================
train_model(model, criterion, optimizer, train_loader, val_loader, epochs=config.epochs)


Epoch 1/50
------------------------------
[Batch 10/251] Loss: 1.3955, Batch Acc: 0.3125
[Batch 20/251] Loss: 1.2868, Batch Acc: 0.8125
[Batch 30/251] Loss: 1.2712, Batch Acc: 0.6250
[Batch 40/251] Loss: 1.1053, Batch Acc: 0.8125
[Batch 50/251] Loss: 1.1221, Batch Acc: 0.6250
[Batch 60/251] Loss: 0.9547, Batch Acc: 0.6875
[Batch 70/251] Loss: 1.1033, Batch Acc: 0.6250
[Batch 80/251] Loss: 0.6963, Batch Acc: 0.9375
[Batch 90/251] Loss: 0.5833, Batch Acc: 0.8750
[Batch 100/251] Loss: 0.8731, Batch Acc: 0.7500
[Batch 110/251] Loss: 0.6161, Batch Acc: 0.8750
[Batch 120/251] Loss: 0.7336, Batch Acc: 0.6875
[Batch 130/251] Loss: 0.5223, Batch Acc: 0.9375
[Batch 140/251] Loss: 1.0451, Batch Acc: 0.6250
[Batch 150/251] Loss: 0.5837, Batch Acc: 0.8750
[Batch 160/251] Loss: 0.7721, Batch Acc: 0.6875
[Batch 170/251] Loss: 0.7120, Batch Acc: 0.6875
[Batch 180/251] Loss: 0.7182, Batch Acc: 0.7500
[Batch 190/251] Loss: 0.3602, Batch Acc: 1.0000
[Batch 200/251] Loss: 0.4935, Batch Acc: 0.8750
[Batch